# Part 3.4. CNN

## Prepare embedding matrix and metadata

In [10]:
import json
from pathlib import Path

import numpy as np

# Load the embedding matrix that handled OOV words
embedding_path = Path("models/w2v_matrix.npy")
index_from_word_path = Path("models/w2v_index.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

## Prepare dataset

In [11]:
from utils.text import tokenize
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = tokenize(dataset["train"])
val_dataset = tokenize(dataset["validation"])
test_dataset = tokenize(dataset["test"])

In [12]:
from utils.text import token_to_index

train_dataset = token_to_index(dataset=train_dataset, index_from_word=index_from_word)
val_dataset = token_to_index(dataset=val_dataset, index_from_word=index_from_word)
test_dataset = token_to_index(dataset=test_dataset, index_from_word=index_from_word)

In [13]:
train_dataset = train_dataset.select_columns(["label", "original_len", "indexes"])
val_dataset = val_dataset.select_columns(["label", "original_len", "indexes"])
test_dataset = test_dataset.select_columns(["label", "original_len", "indexes"])

In [14]:
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

In [6]:
train_dataset

Dataset({
    features: ['label', 'original_len', 'indexes'],
    num_rows: 8530
})

## Train CNN Model

In [7]:
SEARCH_SPACE = {
    "batch_size": [512, 1024, 2048],
    "learning_rate": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    "optimizer_name": ["Adam"],
    # CNN Model Parameters
    "dropout": [0.1, 0.3, 0.5, 0.7, 0.9],
    "hidden_dim": [600, 500, 400, 300],
    "n_grams": [
        [2],
        [3],
        [4],
        [5],
        [6],
        [2, 3],
        [2, 3, 4],
        [2, 3, 4, 5],
        [2, 3, 4, 6],
        [3, 4],
        [3, 4, 5],
        [3, 4, 6],
        [4, 5],
        [4, 5, 6],
        [5, 6],
    ],
}

In [ ]:
import optuna

from utils.train import (
    CNNArgs,
    DataArgs,
    OptimizerArgs,
    train_cnn_model_with_parameters,
)


def objective(trial: optuna.Trial):
    batch_size = trial.suggest_categorical("batch_size", SEARCH_SPACE["batch_size"])
    learning_rate = trial.suggest_categorical(
        "learning_rate", SEARCH_SPACE["learning_rate"]
    )
    optimizer_name = trial.suggest_categorical(
        "optimizer_name", SEARCH_SPACE["optimizer_name"]
    )
    # CNN Model Parameters
    dropout = trial.suggest_categorical("dropout", SEARCH_SPACE["dropout"])
    hidden_dim = trial.suggest_categorical("hidden_dim", SEARCH_SPACE["hidden_dim"])
    n_grams = trial.suggest_categorical("n_grams", SEARCH_SPACE["n_grams"])

    log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; n_grams_{"_".join(map(str, n_grams))}; dropout_{dropout}  ----------"
    print(log_message)

    cnn_args = CNNArgs(
        embedding_matrix=embedding_matrix,
        freeze_embedding=False,
        hidden_dim=hidden_dim,
        dropout=dropout,
        n_grams=n_grams,
    )

    optimizer_args = OptimizerArgs(
        optimizer_name=optimizer_name,
        learning_rate=learning_rate,
    )

    data_args = DataArgs(
        batch_size=batch_size,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
    )

    val_acc = train_cnn_model_with_parameters(
        data_args=data_args,
        cnn_args=cnn_args,
        optimizer_args=optimizer_args,
    )

    return val_acc


# Set up the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150)

In [3]:
from utils.analytics import load_tensorboard_logs

df = load_tensorboard_logs("tb_logs/cnn").sort_values("val_acc", ascending=False)
df.head(10)

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,n_grams,dropout,epoch,train_loss,val_loss,filename
10,0.781426,0.930071,2048,500,0.001,Adam,2,0.3,13.0,0.239519,0.481192,events.out.tfevents.1730676175.yuriarch.2971.10
23,0.778612,0.898188,2048,500,0.001,Adam,2_3_4_5,0.7,28.0,0.301202,0.487332,events.out.tfevents.1730650766.yuriarch.53982.9
47,0.776735,1.000000,2048,500,0.010,Adam,2,0.3,9.0,0.001249,0.484529,events.out.tfevents.1730676650.yuriarch.2971.14
2,0.775797,0.822514,2048,500,0.001,Adam,2,0.9,52.0,0.378091,0.482208,events.out.tfevents.1730675393.yuriarch.2971.5
17,0.773921,0.865978,2048,500,0.001,Adam,4,0.7,26.0,0.321847,0.495995,events.out.tfevents.1730651270.yuriarch.53982.12
13,0.770169,0.875180,2048,400,0.001,Adam,2_3_4_5,0.7,28.0,0.288041,0.478942,events.out.tfevents.1730651947.yuriarch.99853.3
5,0.770169,0.840097,2048,600,0.001,Adam,3,0.5,17.0,0.368825,0.488717,events.out.tfevents.1730675616.yuriarch.2971.6
43,0.769231,0.944701,2048,500,0.001,Adam,3_4,0.1,15.0,0.262486,0.491361,events.out.tfevents.1730649767.yuriarch.53982.2
19,0.769231,0.988132,2048,500,0.010,Adam,2_3_4_6,0.3,11.0,0.052200,0.489992,events.out.tfevents.1730677377.yuriarch.2971.19
14,0.766417,0.878448,2048,300,0.001,Adam,3,0.3,18.0,0.415630,0.497469,events.out.tfevents.1730650200.yuriarch.53982.5


In [4]:
best_cnn_model_configuration = df.head(1)
best_cnn_model_configuration

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,n_grams,dropout,epoch,train_loss,val_loss,filename
10,0.781426,0.930071,2048,500,0.001,Adam,2,0.3,13.0,0.239519,0.481192,events.out.tfevents.1730676175.yuriarch.2971.10


## Best Config on Test set

In [5]:
from pathlib import Path
from models.CNN import CNNClassifier


best_cnn_model_file_name = best_cnn_model_configuration["filename"].item()
matched_files = list(Path().rglob(best_cnn_model_file_name))

if not matched_files:
    print("Model checkpoint not found!")
else:
    checkpoint_dir = matched_files[0].parent / "checkpoints"
    checkpoint_files = (
        list(checkpoint_dir.glob("*.ckpt")) if checkpoint_dir.exists() else []
    )

    if not checkpoint_files:
        print("No checkpoint files found in the checkpoint directory!")
    else:
        best_checkpoint = checkpoint_files[0]
        best_cnn_model = CNNClassifier.load_from_checkpoint(
            best_checkpoint
        )
        print(best_cnn_model)

CNNClassifier(
  (model): CNN(
    (embedding): Embedding(16334, 300, padding_idx=0)
    (dropout): Dropout(p=0.3, inplace=False)
    (fc): Linear(in_features=500, out_features=250, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=250, out_features=2, bias=True)
  )
  (metric): MulticlassAccuracy()
)


/home/yuri/sc4002-nlp-sentiment-classification/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'cnn_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['cnn_model'])`.


In [15]:
import lightning as L
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_dataset, shuffle=True)

trainer = L.Trainer(accelerator="cpu")
trainer.test(best_cnn_model, test_dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Testing DataLoader 0: 100%|██████████| 1066/1066 [00:02<00:00, 445.65it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7879924774169922
        test_loss           0.4289623200893402
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.4289623200893402, 'test_acc': 0.7879924774169922}]